In [1]:
#############################################################################
# regular express를 이용한 말뭉치 정재하기
#
#############################################################################
# 정규화할 인자들
alphabetP=True            # False=알바벳 제거
numberP=True             # False=숫자제거
punctuationP=False       # False=특수문자 (, . ? !) 제거
symbolP=False            # False=괄호((), [], {}, ], ')등 제거

# 4번이상 반복되는 단어 중복 제거 (remove_repeat_wordP=2 : 2번만 호출)
# => 예: 안녕안녕안녕안녕 => 안녕안녕, 안녕 안녕 안녕 안녕 => 그대로(*띄어쓰기한 단어는 중복제거 안됨)
remove_repeat_wordP=2     

# 4번이상 반복되는 문자 중복 제거 (nremove_repeat_charP=2 : 2번만 호출)
# => 예: aaaaa => aa, aaa aaa aaa aaa => 그대로(*띄어쓰기한 문자는 중복제거 안됨)
remove_repeat_charP=2

# 정규화할 입력 text
in_txt_path = '../../../korpora/kowiki/kowiki-202206-nlp-corpus.txt'
#in_txt_path = '../../../korpora/moco/test-60.txt'

# 출력 text
out_txt_path = '../../../korpora/kowiki/re-kowiki-202206-nlp-corpus.txt'

# 삭제할 최대 길이 문장
remove_min_len = 30 # 20이면 20보다 작은문장은 제거함

remove_max_len = 600 # 1000자 보다 큰 문장은 제거함

print_count = 100000 # 10000번째마다 문장 출력해봄

In [2]:
# reqular 함수들 정의 
import re

doublespace_pattern = re.compile('\s+')
repeatchars_pattern = re.compile('(\w)\\1{3,}')
number_pattern = re.compile('[0-9]')
punctuation_pattern = re.compile('[,\.\?\!]')
symbol_pattern = re.compile('[()\[\]\{\}`]')
hangle_pattern = re.compile('[ㄱ-ㅎㅏ-ㅣ가-힣]')
alphabet_pattern = re.compile('[a-zA-Z]')

hangle_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣]')
hangle_number_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]')
text_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9,\.\?\!\"\'-()\[\]\{\}]')

##========================================================================
# 4번이상 반복되는 단어 중복 제거 (num_repeat=2 : 2번만 호출)
# => 예: 안녕안녕안녕안녕 => 안녕안녕, 안녕 안녕 안녕 안녕 => 그대로(*띄어쓰기한 단어는 중복제거 안됨)
repeatchars_patterns = [
    re.compile('(\w\w\w\w)\\1{3,}'),
    re.compile('(\w\w\w)\\1{3,}'),
    re.compile('(\w\w)\\1{3,}'),
    re.compile('(\w)\\1{3,}')
]

def normalizetoken(sentence, num_repeat=2):
    tokens = sentence.split()
    return ' '.join(_normalize_korean_token(token, num_repeat) for token in tokens)

def _normalize_korean_token(token, num_repeat=2):
    #print(token)
    #token = _normalize_emoji(token)
    token = _remove_repeat(token, num_repeat)
    return token

def _remove_repeat(token, num_repeat=2):
    if num_repeat > 0:
        for pattern in repeatchars_patterns:
            #print(pattern)
            token = pattern.sub('\\1' * num_repeat, token)
    return token
##========================================================================


def normalize(sentence, 
              alphabet=False, 
              number=False,
              punctuation=False, 
              symbol=False, 
              remove_repeat_word=0, 
              remove_repeat_char=0):

    sentence = text_filter.sub(' ', sentence)
    
    if not alphabet:
        sentence = alphabet_pattern.sub(' ', sentence)
    if not number:
        sentence = number_pattern.sub(' ', sentence)
    if not punctuation:
        sentence = punctuation_pattern.sub(' ', sentence)
    if not symbol:
        sentence = symbol_pattern.sub(' ', sentence)
    if remove_repeat_char > 0:
        sentence = repeatchars_pattern.sub('\\1' * remove_repeat_char, sentence)
        
    if remove_repeat_word > 0:
        sentence = normalizetoken(sentence, num_repeat=remove_repeat_word)
        
    return doublespace_pattern.sub(' ', sentence).strip()

In [3]:
## yield 를 이용하여 generator 함수 정의    
def get_generator_corpus(data, max_len: int=100000):
    
    #dataset = data
    dataset = list(set(data)) # ****중복 문장 제거(*순서 유지 안함)
    
    for start_idx in range(0, len(dataset), max_len):
        samples = dataset[start_idx : start_idx + max_len]
        yield samples

In [4]:
from tqdm.notebook import tqdm

# 1.분석할 text 읽어오기(한줄씩 읽어오기)
with open(in_txt_path, 'r', encoding='utf8') as f:
    #lines = f.read()
    lines = f.readlines()
    #print(lines)
        
training_corpus = get_generator_corpus(lines)
        
result=[]
count = 0
for lines in training_corpus:
    for line in tqdm(lines):
        
        #print(line)
        # false=제거함.
        clearline = normalize(line, 
                              alphabet=alphabetP, number=numberP, punctuation=punctuationP, symbol=symbolP, 
                              remove_repeat_word=remove_repeat_wordP, remove_repeat_char=remove_repeat_charP)
        
        clearline1=clearline.strip()
        count += 1
        if count % print_count == 0:  # print_count 째마다 출력해봄
            print(f'[{count}]{clearline1}')

        #if clearline1 not in result: # ****순서유지 중복 제거(*순서유지하는 경우 시간 오래걸림)

        # remove_min_len 보가 크고, remove_max_len 보다 작은 경우에만 저장
        clearline1_len = len(clearline1)
        if clearline1_len > remove_min_len and clearline1_len < remove_max_len: # 20자 이상인 경우에만 출력
            #print(clearline1)
            result.append(clearline1)
    
# 중복 문장 제거(*순서 유지 안함)
#result = list(set(clearlinelist))

print('\r\n===[out]====')
print(f'*len:{len(result)}')
count = 0
for line1 in result:
    print(line1)
    count += 1
    if count > 10:
        break

  0%|          | 0/100000 [00:00<?, ?it/s]

[100000]종전 패자조 결승과 최종 결승에 적용되었던 5전 3선승제를 패자 준결승전 승자결승부터 적용했다


  0%|          | 0/100000 [00:00<?, ?it/s]

[200000]또 2006년 12월에 내부를 다시 개장했다


  0%|          | 0/100000 [00:00<?, ?it/s]

...

[5900000]It arranges for the command to ignore TERM signals


  0%|          | 0/100000 [00:00<?, ?it/s]

[6000000]2012년 하계 올림픽 감비아 선수단은 영국 런던에서 개최된 2012년 하계 올림픽에 참가한 올림픽 감비아 선수단이다


  0%|          | 0/57204 [00:00<?, ?it/s]


===[out]====
*len:5334644
인쇄 차단 알림 인쇄 차단된 경우 차단된 프로그램과 파일명 등을 알림 메시지로 보여줍니다
예를 들어 아오 역에서 오사카난바 역까지의 연락 승차권은 구입할 수 없기 때문에 고베 전철 쪽에서 니시쿠조를 지나 승차하는 경우에는 일단 승차역에서 니시쿠조까지의 연락 승차권을 구입해 내리는 역의 정산기에서 부족한 운임을 정산해야 한다
수리논리학에서 2차 논리 는 임의의 다항 관계 및 다항 연산에 대한 변수 및 이에 대한 전칭 존재 기호를 사용할 수 있는 논리이다
이러한 애트리뷰트를 작성한 후 선택한 애트리뷰트에 대해 다음 설명 양식을 작성하십시오
또한 단일 로그인 기능은 기존 인증 서비스와 통합되어야 하기 때문에 인증 Authentication 서비스와 긴밀하게 관련되어 있습니다
2015년 기준으로 이 시리즈는 199억 달러의 가치가 있는 것으로 추정된다
그녀는 단체 종합에서 금메달을 획득했고 개인 종합에서 동메달을 땄다
여전히 공연 무대에 서고 새 음반도 녹음에 들어갔지만 1990년 11월 1일 김현식은 서울 동부이촌동 자택에서 간경화로 사망했다
몸 때문에 생긴 그림자의 부재 가는 선으로 테두리가 둘러쳐진 '얕게' 채색된 화면 이례적인 원근법 풍경 속에서 아주 작게 묘사된 인물들이 그런 원칙들이었다
2부리그에서 1위로 13년 만에 1부리그 복귀에 성공시켰다
AMQ7902 The data conversion exit 3 was notloaded


In [5]:
# 리스트를 파일로 저장
with open(out_txt_path, 'w', encoding='utf8') as f:
    f.write('\n'.join(result))